<a href="https://colab.research.google.com/github/aviadarn/AiNotebooks/blob/main/Simple_Video_Analysis_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini 2.5 Pro: Simple Video Analysis Demo



In [1]:
!pip install -q yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.5 MB/s eta 0:00:00


In [29]:
# @title Download 10s Clip
video_url = 'https://www.youtube.com/watch?v=YouqODxBsc0' # @param {type:"string"}
output_filename = 'video_clip.mp4'

# --download-sections "*00:00-00:10" downloads the first 10 seconds
# --force-keyframes-at-cuts ensures the cut is accurate by re-encoding if needed
!yt-dlp --download-sections "*0-30" --force-keyframes-at-cuts -f "best[ext=mp4]" -o "{output_filename}" "{video_url}"

print(f"Downloaded: {output_filename}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=YouqODxBsc0
[youtube] YouqODxBsc0: Downloading webpage
[youtube] YouqODxBsc0: Downloading android sdkless player API JSON
[youtube] YouqODxBsc0: Downloading web safari player API JSON
[youtube] YouqODxBsc0: Downloading m3u8 information
[info] YouqODxBsc0: Downloading 1 format(s): 96
[info] YouqODxBsc0: Downloading 1 time ranges: 0.0-30.0
[download] video_clip.mp4 has already been downloaded
[download] 100% of    2.40MiB
Downloaded: video_clip.mp4


In [3]:
from IPython.display import Video
import os

if os.path.exists(output_filename):
    display(Video(output_filename, embed=True, width=600))
else:
    print("Video file was not created.")

In [23]:
# @title 1. Setup: Import Libraries & Configure API Key
import time
from google.colab import userdata
from google import genai
from IPython.display import Markdown

# Configure API Key (replace 'GOOGLE_API_KEY' with your key name in Colab secrets)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

# Define the model
MODEL_NAME = 'models/gemini-2.5-flash'
print(f"Using model: {MODEL_NAME}")


Using model: models/gemini-2.5-flash


In [24]:
# @title 2. Upload Files: Video and Reference Text
def upload_file(client, file_name):
    print(f"Uploading file: {file_name}...")
    try:
        file_obj = client.files.upload(file=file_name)
        print(f"Completed upload: {file_obj.uri}")
        return file_obj
    except FileNotFoundError:
        print(f"Error: File '{file_name}' not found. Please upload it to Colab.")
        return None
    except Exception as e:
        print(f"Error uploading file: {e}")
        return None

# File names (change these to your actual files)
video_file_name = 'video_clip.mp4'
reference_file_name = 'reference.txt'

# Upload both files
video_file = upload_file(client, video_file_name)
# ref_file = upload_file(client, reference_file_name)

# Ensure both files are uploaded
if not video_file:
    raise ValueError("One or both files failed to upload. Please check and try again.")

Uploading file: video_clip.mp4...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/cde7qy7b2rrb


In [25]:

# @title 3. Wait for Files to Process
def wait_for_file(client, file_obj):
    if not file_obj:
        return
    print(f"Processing {file_obj.name}", end='')
    while file_obj.state.name == "PROCESSING":
        print('.', end='', flush=True)
        time.sleep(5)  # Check every 5 seconds
        file_obj = client.files.get(name=file_obj.name)
    print(f" Done. State: {file_obj.state.name}")
    if file_obj.state.name == "FAILED":
        raise ValueError(f"File processing failed for {file_obj.name}")

# Wait for both files
wait_for_file(client, video_file)

Processing files/cde7qy7b2rrb. Done. State: ACTIVE


In [26]:
# @title 4. Define the Analysis Prompt
prompt = """
        You are a video-analysis model specialized in brand and logo detection.

        TASK:
        Analyze the entire video provided as input. Identify every visible or partially visible brand, product label, logo, or trademark—even if it appears only briefly, is blurred, small, in the background, or partially occluded.

        WHAT TO RETURN:
        Return a JSON object with the following for each brand detected:
        - brand: the detected brand name
        - timestamp_start: exact appearance start time (HH:MM:SS.mmm)
        - timestamp_end: exact disappearance time (HH:MM:SS.mmm)
        - confidence: confidence level (0–100)
        - description: short description of where the brand appears in the scene

        REQUIREMENTS:
        - Extract precise timestamps.
        - Include only verifiable brands—no hallucinations.
        - If unsure, mark confidence < 50 or exclude.
        - If no brands are detected, return: { "brands_detected": [] }

        OUTPUT FORMAT (STRICT):
        {
          "brands_detected": [
            {
              "brand": "string",
              "timestamp_start": "HH:MM:SS.mmm",
              "timestamp_end": "HH:MM:SS.mmm",
              "confidence": number,
              "description": "string"
            }
          ]
        }

        Begin analysis once the video is provided.
    """

In [27]:
# @title 5. Generate Analysis
print("Generating analysis (this may take a moment)...")
try:
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=[ video_file, prompt])
    print("Analysis complete.")
except Exception as e:
    print(f"Error generating analysis: {e}")
    response = None

Generating analysis (this may take a moment)...
Analysis complete.


In [28]:

# @title 6. Display Results
if response:
    display(Markdown(response.text))
else:
    print("No analysis results generated.")

```json
{
  "brands_detected": [
    {
      "brand": "GOLF HOLIDAYS DIRECT",
      "timestamp_start": "00:00:00.000",
      "timestamp_end": "00:00:09.916",
      "confidence": 95,
      "description": "Text 'GOLF HOLIDAYS DIRECT' is clearly visible on the side of a golf bag positioned on the left side of the screen."
    }
  ]
}
```